# Calcular score das avaliações

In [1]:
import numpy as np
import pandas as pd

In [2]:
# declare a list tasks whose products you want to use as inputs
upstream = None
product = None
df_input_path = '../../dados/X_trainToronto.csv'
df_review_input_path = '../../dados_produzidos/reviewsTrainTorontoSentiment.csv'

In [3]:
# Parameters
df_input_path = "../../dados/X_trainToronto.csv"
df_review_input_path = "../../dados_produzidos/reviewsTrainTorontoSentiment.csv"
product = {
    "data": "/home/guilherme/\xc1rea de Trabalho/Estudo Faculdade/Disciplinas/minera\xe7\xe3o de dados/Desafio/pipelines/processamento_features/output/calcular_score_reviews_data.parquet",
    "nb": "/home/guilherme/\xc1rea de Trabalho/Estudo Faculdade/Disciplinas/minera\xe7\xe3o de dados/Desafio/pipelines/processamento_features/output/output_calcular_score_reviews.ipynb",
}


In [4]:
df_reg = pd.read_csv(df_input_path)
df_reviews = pd.read_csv(df_review_input_path)
df_reg = df_reg[['business_id', 'review_count']]
df_reg

,business_id,review_count
0,vHzWmPWHN4J1hRR3W3AMQg,99
1,15to24Q-otAHmto7FzsWRg,3
2,8aqKdf4G4AAir8k_Kdslvg,18
3,uxU1vr5AhhkTQ83X0bpeyg,3
4,f702hTJoqdR34Jn23C7d1A,3
...,...,...
17577,Zp5xil8pC6Bm-fFOWIbh0Q,16
17578,o2811xZ0ufRi7WcsqWiGzw,4
17579,v9T9m_TbpLuaC2Kud5UbLA,8
17580,bzGnAIe9OiBHGWJE-dZ_8g,9


## Calcular score das analises

In [5]:
from sklearn.preprocessing import StandardScaler

def make_df_score(df, df_reviews):
    weights = {
        'useful': 1,
        'cool': 0.75,
        'funny': 0.5
    }
    df_reviews['weight'] = df_reviews.apply(lambda row: 1 + row['useful'] * weights['useful'] + row['cool'] * weights['cool'] + row['funny'] * weights['funny'], axis=1)
    def wavg_label(group):
        d = group['label']
        w = group['weight']
        return (d * w).sum() / w.sum()
    
    def wavg_score(group):
        d = group['score']
        w = group['weight']
        return (d * w).sum() / w.sum()
        
    df_reviews_label_grouped = df_reviews.groupby(['business_id']).apply(wavg_label, include_groups=False)
    df_reviews_score_grouped = df_reviews.groupby(['business_id']).apply(wavg_score, include_groups=False)
    df_reviews_scores = pd.DataFrame(df_reviews_label_grouped, columns=['stars']).join(pd.DataFrame(df_reviews_score_grouped, columns=['score']))
    df_score = df.join(df_reviews_scores)
    scaler = StandardScaler()
    df_score['stars'] = scaler.fit_transform(df_score[['stars']])
    return df_score

df_score = make_df_score(df_reg.set_index('business_id'), df_reviews)
df_reg[['stars', 'score']] = df_score.reset_index()[['stars', 'score']]
df_reg[['stars', 'score']]

,stars,score
0,-0.080982,0.532271
1,0.841286,0.577041
2,-1.153910,0.706820
3,0.288943,0.400056
4,-2.627026,0.511156
...,...,...
17577,-1.241298,0.567692
17578,0.757598,0.757226
17579,-0.147039,0.657578
17580,-2.010238,0.696177


In [6]:
df_reg.to_parquet(product['data'])